In [1]:
import pandas as pd
import IPython.display

In [81]:
def to_html_table(results):
    res  = "<table>"
    res += '''<tr><th>Camera</th><th>Resolution</th>
                  <th colspan=\"7\" style=\"text-align:center\">Included features</th>
                  <th colspan=\"3\" style=\"text-align:center\">Frames / s</th></tr>'''
    res += "<tr>"
    for col in ["", "", "Joints", "Skin", "FlexSkin", "Textures", "Fog", "Shadows", "Reflections"]:
        res += f"<td><span style=\"writing-mode: sideways-lr;\">{col}</span></td>"
    res += '''<td style=\"text-align:left\">Local desktop<br />GTX 1660<br /><em>glfw</em></td>
              <td style=\"text-align:left\">Local desktop<br />GTX 1660<br /><em>egl</em></td>
              <td style=\"text-align:left\">Cluster<br /><tt>olveczkygpu</tt><br />V100 16GB<br /><em>egl</em></td>'''
    res += "</tr><tbody>"
    for r in results.itertuples():
        res += "<tr>"
        res += f"<td>{r.camera}</td>"
        res += f"<td>{r.res} x {r.res}</td>"
        #res += f"<td>{r.backend}</td>"
        for visFeature in ['visJoint', 'visSkin', 'visFlexSkin', 'visTexture', 'visFog', 'visShadow', 'visReflection']:
            symbol = "✅" if getattr(r, visFeature) else "❌"
            res += f"<td>{symbol}</td>"
        for fps in (r.fps, r.fps_egl, r.fps_cluster):
            res += f'''<td style=\"text-align:left\">
                            <div style='display: inline-block;
                                        background-color: darkblue;
                                        width: {(fps/20):.1f}px;
                                        height: 15px;
                                        margin-right: 2px;
                                        margin-top: 2px;
                                        margin-bottom: 0px;'></div>
                            {fps:.2f}
                        </td>'''
        res += "</tr>"
    res += "</tbody></table>"
    return IPython.display.HTML(res)

In [82]:
results_local_glfw = pd.read_csv("results/feature_benchmarking_local.csv")
results_local_egl = pd.read_csv("results/feature_benchmarking_local_egl.csv")
results_cluster = pd.read_csv("results/feature_benchmarking_olveczkygpu.csv")

In [83]:
results_combined = results_local_glfw.assign(fps_egl=results_local_egl.fps).assign(fps_cluster=results_cluster.fps)

In [84]:
full_html = "<html><head><title>Rendering benchmark</title>"
full_html += '''<style>
                    body {
                        font-family: system-ui, -apple-system, blinkmacsystemfont, "Segoe UI", helvetica, arial, sans-serif, "Apple Color Emoji", "Segoe UI Emoji", "Segoe UI Symbol";
                    }
                    td, th, tr {
                        margin: 0px;
                        padding: 6px;
                    }
                    tr:nth-child(odd) {
                        background: #EEE
                    }
                    table {
                        border-collapse: collapse;
                    }
                </style>'''
full_html += "</head><body>"
full_html += "<h1>Side view</h1>"
full_html += "<img src=\"example_renderings_local_side_256x256.png\" width=\"800\"/>"
full_html += to_html_table(results_combined.query("camera == 'side'")).data
full_html += "<h1>Egocentric</h1>"
full_html += "<img src=\"example_renderings_local_egocentric_256x256.png\" width=\"800\"/>"
full_html += to_html_table(results_combined.query("camera == 'egocentric' & res == 256")).data
full_html += "<h1>Egocentric, low-res</h1>"
full_html += "<img src=\"example_renderings_local_egocentric_64x64.png\" width=\"800\"/>"
full_html += to_html_table(results_combined.query("camera == 'egocentric' & res == 64")).data
full_html += "</body></html>"

In [85]:
with open("plots/benchmarking_report.html", "w") as f:
    f.write(full_html)

In [32]:
IPython.display.HTML(full_html)